# Link Trinotate annotations and Ensembl data to contig names for transcriptome

In [1]:
tab1 = open('~/WRASSE/pasaA_annotation/pasaA_trinotate_annotation_report.xls', 'r')

In [2]:
# dict of dicts
zfish = {}
zfishX = {}
zlist = []
zlistX = []
for line in tab1:
    parts = line.split('\t')
    tran = parts[1]
    
    zpX = parts[7]
    
    zp = parts[9]
    if zp.startswith('E'):
        zpC = zp.split('^')[0]
        zfish[tran]=zpC
        zlist.append(zpC)
    if zpX is not '.': #.startswith('E'):
        zpCx = zpX.split('^')[0]
        zfishX[tran]=zpCx
        zlistX.append(zpCx)

print(len(zfish))
print(len(zfishX))
print(len(zlist))
print(len(zlistX))
tab1.close()

17051
27062
17496
28927


In [3]:
zfish['asmbl_20']

'ENSDARP00000066269.3'

In [4]:
zset = set(zlist)
zsetX = set(zlistX)

In [5]:
zsetDiff = zsetX - zset

In [6]:
len(zsetDiff)

4579

In [7]:
zsetDiffX = zset - zsetX

In [8]:
len(zsetDiffX)

862

In [9]:
print(len(zset))
print(len(zsetX))

14091
17808


In [10]:
fullz = zset | zsetX
print(len(fullz))

18670


In [11]:
zfish_asmblP = list(zfish)
print(zfish_asmblP[0])

asmbl_34


## Match asmbl name to transcriptome contig name

In [12]:
contig_map = open('~/WRASSE/pasaA_annotation/bluePasa_1A.pasa_assemblies_described.txt')

In [13]:
contigs = {}
for line in contig_map:
    if line.startswith('#'):
        continue
    else:
        parts = line.split('\t')
        asmbl = parts[2]
        contig = parts[3]
        if ',' in contig: # indicates more than one
            contig_ls = contig.split(',')
            for cn in contig_ls:
                contigs[cn]=asmbl
        else:
            contigs[contig]=asmbl

print(len(contigs))
contig_map.close()

197538


In [36]:
# now can link across contig, asmbl, and peptide
contigs['c34423_g1_i1']

'asmbl_34'

In [23]:
zfishX['asmbl_34']

'ENSDARP00000086301.5'

## Import biomart file, and link peptides to gene

In [16]:
biomart = open('~/WRASSE/Anns_zebrafish_peptide_gene_descriptions.txt')

In [17]:
pep2gene = {}
gene2descr = {}
for line in biomart:
    if line.startswith('#'):
        continue
    else:
        line = line.strip('\n')
        parts = line.split('\t')
        pep = parts[0]
        gene = parts[1]
        descr = parts[2]
        pep2gene[pep]=gene
        gene2descr[gene]=descr
        
print(len(pep2gene))
biomart.close()

14245


Now need to modify matrix and output those with annotation

**first**, though, check old annotation for those thrown out of PASA analysis

## Incorporate previous annotations

In [18]:
eta_file = open('~/WRASSE/ericas_transcript_annotations_edited.txt')

In [19]:
eta_genes = {}
eta_desc = {}
for line in eta_file:
    if line.startswith('>Contig_ID'):
        continue
    else:
        parts = line.split('\t')
        contig = parts[0]
        zID = parts[4]
        zDescr = parts[5]
        if zID == '':
            continue
        else:
            eta_genes[contig]=zID
            eta_desc[zID]=zDescr

print(len(eta_genes))
print(len(eta_desc))
eta_file.close()

41257
21273


In [31]:
eta_genes['c154918_g1_i1_split_1']

'ENSDARP00000111713'

In [35]:
eta_desc['ENSDARP00000111713']

'doublesex and mab-3 related transcription factor 1 [Source:ZFIN;Acc:ZDB-GENE-050511-1]'

In [27]:
# Change previously modified contig names
# c234180_g1_i1 = c154918_g1_i1_split_2
# c154918_g1_i1 = c154918_g1_i1_split_1
# c69648_g1_i1 = c69648_g1_i1-a
# c234181_g1_i1 = c69648_g1_i1-b
oldNames = {'c154918_g1_i1_split_1':'c154918_g1_i1','c154918_g1_i1_split_2':'c234180_g1_i1','c69648_g1_i1-a':'c69648_g1_i1','c69648_g1_i1-b':'c234181_g1_i1','kissr1-melissa':'c234179_g1_i1'}

In [32]:
contigListFile = open('eta_contig_list.txt')

In [33]:
contigList = [] # create annotations for all genes
for line in contigListFile:
    line = line.strip('\n')
    
    contigList.append(line)
print(len(contigList))
contigListFile.close()

230626


In [34]:
'c154918_g1_i1' in contigList

False

In [50]:
# make final isoform annotations dictionary
IsoformAnnots = {} # combined with all possible 
missingPeps = []
cwa1 = []
for con in contigList:
    if con in contigs:
        asmbl = contigs[con]
        if asmbl in zfishX:
            pep = zfishX[asmbl]
        elif asmbl in zfish:
            pep = zfish[asmbl]
        else:
            pep = 'NONE'
            geneID = 'NONE'
            
        if pep is not 'NONE':
            if pep in pep2gene:
                geneID = pep2gene[pep]
                descr1 = gene2descr[geneID]
                descr = descr1.split(' [')[0]
            else:
                missingPeps.append(pep)
        
    else:
        if con in eta_genes:
            geneID = eta_genes[con]
            descr1 = eta_desc[geneID]
            descr = descr1.split(' [')[0]
            
        else:
            geneID = 'NONE'
    if geneID is not 'NONE':
        IsoformAnnots.setdefault(con, {})['name']=geneID
        IsoformAnnots.setdefault(con, {})['description']=descr
        cwa1.append(con)
print(len(IsoformAnnots))
print(len(missingPeps))
print(len(cwa1))

44482
10334
44482


In [51]:
missingSet = set(missingPeps)
print(len(missingSet))
print(missingPeps[0])

4156
ENSDARP00000079716.5


In [52]:
#IsoformAnnots
# correcting IsoformAnnots
for k,v in oldNames.items():
    if k in IsoformAnnots:
        IsoformAnnots[v] = IsoformAnnots[k]
        del IsoformAnnots[k]
print(len(IsoformAnnots))

44482


In [53]:
IsoformAnnots['c154918_g1_i1']

{'description': 'doublesex and mab-3 related transcription factor 1',
 'name': 'ENSDARG00000007349'}

In [54]:
cwa = []
for c in cwa1:
    if c in oldNames:
        c = oldNames[c]
    cwa.append(c)
print(len(cwa))

44482


## Output Zebrafish annotations to table

In [60]:
isoAnnotOut = open('Anns_Isoform_zebrafish_annotations','w')

In [61]:
isoAnnotOut.write('contig\tZfish_gene\tZfish_description\n')

36

In [62]:
for con in cwa:
    sprotD = IsoformAnnots[con]
    isoAnnotOut.write(con+'\t'+sprotD['name']+'\t'+sprotD['description']+'\n')
isoAnnotOut.close()

## Create Gene-only set

In [63]:
len(cwa)

44482

In [64]:
GenesIso = {}

for con in cwa:
    geneParts = con.split('_')
    gene = '_'.join(geneParts[:-1])
    GenesIso.setdefault(gene, []).append(con)
print(len(GenesIso))

44406


In [65]:
multiAnnots = []
multis = []
for k,v in GenesIso.items():
    if len(v) > 1:
        multis.append(k)
        annotls = []
        for iso in v:
            ann = IsoformAnnots[iso]['name']
            annotls.append(ann)
        annset = set(annotls)
        if len(annset) > 1:
            multiAnnots.append(k)
print(len(multis))
print(len(multiAnnots))

54
8


In [66]:
multiAnnots

['c110294_g1',
 'c123938_g1',
 'c151266_g1',
 'c29034_g1',
 'c5120_g1',
 'c70023_g2',
 'c7027_g1',
 'c76790_g1']

In [ ]:
# c29034_g1
# c70023_g2
# None are different or without consequence (e.g. both unknown genes)

In [67]:
GeneAnnots = {}
genelist = []
for con in cwa:

    geneparts = con.split('_')
    gene = '_'.join(geneparts[:-1])
    if gene not in genelist: # this adds to time
        genelist.append(gene)
    GeneAnnots[gene]=IsoformAnnots[con]
    
print(len(GeneAnnots))
print(len(genelist))

44406
44406


In [68]:
GeneAnnots['c234180_g1']

{'description': 'doublesex and mab-3 related transcription factor 1',
 'name': 'ENSDARG00000007349'}

## Output Gene Annotations to file

In [69]:
geneAnnotOut = open('Anns_Gene_zebrafish_annotations','w')

In [70]:
geneAnnotOut.write('gene\tzfish_name\tzfish_description\n')

34

In [71]:
for g in genelist:
    sprotD = GeneAnnots[g]
    geneAnnotOut.write(g+'\t'+sprotD['name']+'\t'+sprotD['description']+'\n')

geneAnnotOut.close()
    